In [66]:
import csv
import random

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

In [67]:
# candidate info
candidate_count = 0
candidates = []

# ballot mete data
total_voter_count = 0
total_vote_count = 0
unique_order_count = 0

# ballot data
# map a unique order of ballots to the total count of ballots in this order
 # format: tuple(first choice,second,third choice...): count
ballots = []

ballots_size = []

with open("data/dublin-north.soi", newline="\n") as data_file:
    data = csv.reader(data_file)
    i = 0
    for row in data:
        if i == 0:
            candidate_count = int(row[0])
        elif i <= candidate_count:
            candidates.append(row[1])
        elif i == candidate_count + 1:
            total_voter_count = int(row[0])
            total_vote_count = int(row[1])
            unique_order_count = int(row[2])
        else:
            ballot = tuple(int(j) for j in row[1:])
            ballots.append([ballot, int(row[0])])
            ballots_size.append(len(ballot))
        i += 1

In [68]:
# print(candidate_count)
# for key in ballots.keys():
#     print(key, ballots[key])
#     print(len(key))
# for ballot in ballots:
#     print(ballot, ballots[ballot])
#     assert len(ballot[0]) != 0
# print(train_data[0][:-1])

In [69]:
# unroll ballots
def unroll_ballots(ballots):
    unrolled_ballots = []
    for b in range(len(ballots)):
        for i in range(ballots[b][1]):
            unrolled_ballots.append(list(ballots[b][0]))
    return unrolled_ballots

# unrolled_ballots = unroll_ballots(ballots)
# print(len(unrolled_ballots), " ballots")

In [70]:
# get train and set labels
def known_target_split(unrolled_ballots, target_length):
    known_selections = []
    target_selection = []
    for b in range(len(unrolled_ballots)):
        if len(unrolled_ballots[b]) >= target_length:
            known_selections.append(list(unrolled_ballots[b][:target_length - 1]))
            target_selection.append(unrolled_ballots[b][target_length - 1])
    return known_selections, target_selection

# known_selections, target_selection = known_target_split(unrolled_ballots, 5)
# print(len(known_selections))

In [71]:
# divide into train and test set
# X_train, X_test, y_train, y_test = train_test_split(known_selections,target_selection, test_size = 0.2, random_state=1)

In [72]:
# Create a classifier: a support vector classifier
# clf = svm.SVC(gamma=0.15)

In [73]:
# clf.fit(X_train, y_train)

In [74]:
# predicted = clf.predict(X_test)

In [75]:
# print(f"{metrics.classification_report(y_test, predicted)}\n")

In [79]:
def train_svm(known_selections, target_selection):
    clf = svm.SVC(gamma=0.15)
#     X_train, X_test, y_train, y_test = train_test_split(known_selections,target_selection, test_size = 0.2, random_state=1)    
#     clf.fit(X_train, y_train)
    clf.fit(known_selections, target_selection)
    return clf

In [82]:
# Return an order of ballot as a tuple
def predict(ballots, target_ballot, model):
    # finish this
    
    target = model.predict([target_ballot])
    improved_target_ballot = list(target_ballot)
    improved_target_ballot.append(target[0])
    
#     ##################### place holder ####################
#     temp = list(target_ballot)
#     temp.append(random.randint(1,12))
#     improved_target_ballot = tuple(temp)
#     #######################################################
    
    return improved_target_ballot

In [84]:
# this code will modify the data in "ballots". Need to rerun above data reading code to restore
# data in the dictionary.
for i in range(1, candidate_count):
    # unroll ballots
    unrolled_ballots = unroll_ballots(ballots)
    
    # split into data and labels, where the selection at i + 1 place is the label, selections before
    # are the data
    known_selections, target_selection = known_target_split(unrolled_ballots, i + 1)
    
    # train model for predicting the i+1 preference
    model = train_svm(known_selections, target_selection)
    
    # update ballots with only i preference
#     keys = list(ballots.keys())
    for b in range(len(ballots)):
        ballot = ballots[b][0]
        if len(ballot) == i : #and ballots[b][0] != 0:
            improved_ballot = predict(ballots, ballot, model)
            ballots[b][0] = improved_ballot
#             if improved_ballot in ballots:
#                 ballots[improved_ballot] += ballots[ballot]
#             else:
#                 ballots[improved_ballot] = ballots[ballot]
#             del ballots[ballot]
#             ballots[ballot] = 0 # set old count to 0, should then have no impact on final vote count
# print(ballots)

In [85]:
# Return a dictionary mapping candidate number to score.
# Calculate candidate scores based on Borda's Rule.
def borda_rule_result(ballots,ballots_size=None,score_decay=1):
    candidate_scores = dict.fromkeys(range(1, candidate_count+1), 0)
#     print(candidate_scores)
    for b in range(len(ballots)):
        ballot = ballots[b][0]
        count = ballots[b][1]
        for i in range(candidate_count):
            if not ballots_size is None and ballots_size[b] < i:
                candidate_scores[ballot[i]] += (candidate_count - 1 - i) * count * score_decay
            else:
                candidate_scores[ballot[i]] += (candidate_count - 1 - i) * count
    return candidate_scores

In [ ]:
print(borda_rule_result(ballots, ballots_size,1))

In [ ]:
def svt(ballots, ballots_size=[],score_decay=1):
    elim = [] # eliminated candidates
    while True:
        candidate_scores = dict.fromkeys(range(1, candidate_count+1), 0)
        for b in range(len(ballots)):
            ballot = ballots[b][0]
            count = ballots[b][1]
            for i in range(candidate_count):
                if ballot[i] not in elim:
                    if not ballots_size is None and ballots_size[b] < i:
                        candidate_scores[ballot[i]] += count * score_decay
                    else:
                        candidate_scores[ballot[i]] += count
                    break
        least = 0
        first = True
        for (i,v) in candidate_scores.items():
            if i not in elim:
                if first:
                    least = i
                    first = False
                elif candidate_scores[least] > v:
                    least = i
        # eliminate lowest
        print("Eliminate "+str(least))
        elim.append(least)
        # if 1 left, return as winner
        if len(elim) == candidate_count-1: return [x for x in range(1,candidate_count+1) if x not in elim][0]

In [ ]:
print(svt(ballots, ballots_size, 0))

In [ ]:
def irv(ballots, ballots_size=[],score_decay=1):
    elim = [] # eliminated candidates
    while True:
        candidate_scores = dict.fromkeys(range(1, candidate_count+1), 0)
        for b in range(len(ballots)):
            ballot = ballots[b][0]
            count = ballots[b][1]
            for i in range(candidate_count):
                if ballot[i] not in elim:
                    if not ballots_size is None and ballots_size[b] < i:
                        candidate_scores[ballot[i]] += count * score_decay
                    else:
                        candidate_scores[ballot[i]] += count
                    break
        least = 0
        best = 0
        first = True
        for (i,v) in candidate_scores.items():
            if i not in elim:
                if first:
                    least = i
                    best = i
                    first = False
                elif candidate_scores[least] > v:
                    least = i
                elif candidate_scores[best] < v:
                    best = i
        # if majority, return as winner
        if candidate_scores[best] > len(ballots)/2: return best
        # otherwise eliminate lowest and continue
        print("Eliminate "+str(least))
        elim.append(least)
        

In [ ]:
print(irv(ballots, ballots_size, 0))

In [ ]:
def copeland(ballots, ballot_size=None, score_decay=1):
    
    candidate_scores = dict.fromkeys(range(1, candidate_count+1), dict.fromkeys(range(1, candidate_count+1), 0))
    # get dic where [i][j] -> # times i beats j
    for b in range(len(ballots)):
        ballot = ballots[b][0]
        count = ballots[b][1]
        for i in range(candidate_count):
            for j in range(i+1,candidate_count):
                # ballot[i] beats all ballot[j]
                if not ballots_size is None and ballots_size[b] < i:
                    candidate_scores[ballot[i]][ballot[j]] += count * score_decay
                else:
                    candidate_scores[ballot[i]][ballot[j]] += count
                
    
    
    candidate_pairwise_wins = dict.fromkeys(range(1, candidate_count+1), 0)
    
    # loop through all pairwise matchups
    for i in range(1, candidate_count+1):
        for j in range(i+1, candidate_count+1):
            if candidate_scores[i][j] > candidate_scores[j][i]:
                # i beats j
                candidate_pairwise_wins[i] += 1
            else: 
                candidate_pairwise_wins[j] += 1
                
    
    max = 1
    # get candidate with most pairwise wins
    for i in range(1, candidate_count+1):
        if candidate_pairwise_wins[i] > candidate_pairwise_wins[max]:
            max = i
        print(str(i) + " wins: " + str(candidate_pairwise_wins[i]))
    return max
    
    
    

In [ ]:
# Update decay with exopnential...
print(copeland(ballots, ballots_size, 1))

In [ ]:
# from numpy import random
# import numpy as np
def reduce_ballots(ballots, ballots_size, alpha, beta):
    array = np.array(range(len(ballots)))
    random.shuffle(array)
    change_ballots = array.to_list()
    alpha_cnt = int(len(ballots) * alpha)
    for b in change_ballots[0:alpha_cnt]: # Change first alpha random ballots
        # Can we get original data set???
        # Assuming all ballots are complete...
        step = 1
        ballot = ballots[b][0]
        count = ballots[b][1]
        for i in range(candidate_count-1,0,-1):
            r = random.uniform(0,1)
            if r > beta^step:
                ballot = ballot[:-1]
                step += 1
            else:
                break
        ballots[b] = [ballot,count]
        ballots_size[b] = candidate_count - (step-1)
        
    return ballots, ballots_size
        